In [1]:
from src.core import *
from src import DSL
import inspect
import typing

dsl.gets_stab return type hint not valid.
[None, ['dsl.check_move_is_gyro_ball', 'move'], ['dsl.check_move_rainy', 'move'], ['dsl.check_move_sds_always', 'move'], ['dsl.check_move_sds_if_hits_opp', 'move'], ['dsl.check_move_sunny', 'move'], None, ['dsl.move_base_power', 'move', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.move_is_physical', 'move'], ['dsl.move_is_special', 'move'], ['dsl.move_is_status', 'move'], ['dsl.move_type', 'move', <Rule.ENUM_COMPARATOR: 17>, <Rule.POKEMON_TYPE: 18>], ['dsl.opp_base_defense', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.opp_base_spec_defense', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.opp_speed', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.player_base_attack', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.player_base_spec_defense', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.player_base_speed', <Rule.NUM_COMPARATOR: 16>, <Rule.INT_NUM: 15>], ['dsl.type_multiplier', 'move', <Rule.NUM_COMPA

In [2]:
print(GRAMMAR)

{<Rule.START: 1>: [<src.core.Diminishing object at 0x11bc4da10>], <Rule.IF_BLOCK: 2>: [[<Rule.BOOL_EXP: 4>, <Rule.IF_BODY: 3>]], <Rule.IF_BODY: 3>: [<Rule.CHANGE_SCORE: 9>], <Rule.BOOL_EXP: 4>: [<Rule.BOOL: 8>, <Rule.AND_EXP: 5>, <Rule.NOT_EXP: 6>], <Rule.AND_EXP: 5>: [[<Rule.BOOL: 8>, <Rule.BOOL: 8>]], <Rule.NOT_EXP: 6>: [<Rule.BOOL: 8>], <Rule.OR_EXP: 7>: [[<Rule.BOOL: 8>, <Rule.BOOL: 8>]], <Rule.BOOL: 8>: [<Rule.LIB_CALL: 13>], <Rule.CHANGE_SCORE: 9>: [<Rule.SCORE_NUM: 10>], <Rule.SCORE_NUM: 10>: ['-8', '-7', '-6', '-5', '-4', '-3', '-2', '-1', '1', '2', '3', '4', '5', '6', '7', '8'], <Rule.FLOAT_NUM: 14>: ['0.25', '0.5', '1', '2', '4'], <Rule.INT_NUM: 15>: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56'

In [4]:
for seed in range(20):
    tree = get_random_tree(seed)
    script = exec_tree(tree)
    script.tree.print()
    print(script.raw_script)

Rule.START
├── Rule.IF_BLOCK
│   ├── Rule.BOOL_EXP
│   │   └── Rule.AND_EXP
│   │       ├── Rule.BOOL
│   │       │   └── Rule.LIB_CALL
│   │       │       ├── dsl.player_base_attack
│   │       │       ├── Rule.NUM_COMPARATOR
│   │       │       │   └── ==
│   │       │       └── Rule.INT_NUM
│   │       │           └── 55
│   │       └── Rule.BOOL
│   │           └── Rule.LIB_CALL
│   │               ├── dsl.opp_speed
│   │               ├── Rule.NUM_COMPARATOR
│   │               │   └── ==
│   │               └── Rule.INT_NUM
│   │                   └── 70
│   └── Rule.IF_BODY
│       └── Rule.CHANGE_SCORE
│           └── Rule.SCORE_NUM
│               └── -5
├── Rule.IF_BLOCK
│   ├── Rule.BOOL_EXP
│   │   └── Rule.BOOL
│   │       └── Rule.LIB_CALL
│   │           ├── dsl.player_base_speed
│   │           ├── Rule.NUM_COMPARATOR
│   │           │   └── >=
│   │           └── Rule.INT_NUM
│   │               └── 61
│   └── Rule.IF_BODY
│       └── Rule.CHANGE_SCORE
│           └── 